# First Term Project: Cranfield Collection
“The Cranfield collection [...] was the pioneering test collection in allowing CRANFIELD precise quantitative measures of information retrieval effectiveness [...]. Collected in the United Kingdom starting in the late 1950s, it contains 1398 abstracts of aerodynamics journal articles, a set of 225 queries, and exhaustive relevance judgments of all (query, document) pairs.” [1, Section 8.2]

Your tasks, reviewed by your colleagues and the course instructors, are the following:

1.   *Implement an unsupervised ranked retrieval system*, [1, Chapter 6] which will produce a list of documents from the Cranfield collection in a descending order of relevance to a query from the Cranfield collection. You MUST NOT use relevance judgements from the Cranfield collection in your information retrieval system. Relevance judgements MUST only be used for the evaluation of your information retrieval system.

2.   *Document your code* in accordance with [PEP 257](https://www.python.org/dev/peps/pep-0257/), ideally using [the NumPy style guide](https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard) as seen in the code from exercises.  
     *Stick to a consistent coding style* in accordance with [PEP 8](https://www.python.org/dev/peps/pep-0008/).

3.   *Reach at least 22% mean average precision* [1, Section 8.4] with your system on the Cranfield collection. You MUST record your score either in [the public leaderboard](https://docs.google.com/spreadsheets/d/e/2PACX-1vT0FoFzCptIYKDsbcv8LebhZDe_20GFeBAPmS-VyImlWbqET0T7I2iWy59p9SHbUe3LX1yJMhALPcCY/pubhtml) or in this Jupyter notebook. You are encouraged to use techniques for tokenization, [1, Section 2.2] document representation [1, Section 6.4], tolerant retrieval [1, Chapter 3], relevance feedback and query expansion, [1, Chapter 9] and others discussed in the course.

4.   _[Upload an .ipynb file](https://is.muni.cz/help/komunikace/spravcesouboru#k_ss_1) with this Jupyter notebook to the homework vault in IS MU._ You MAY also include a brief description of your information retrieval system and a link to an external service such as [Google Colaboratory](https://colab.research.google.com/), [DeepNote](https://deepnote.com/), or [JupyterHub](https://iirhub.cloud.e-infra.cz/).

[1] Manning, Christopher D., Prabhakar Raghavan, and Hinrich Schütze. [*Introduction to information retrieval*](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf). Cambridge university press, 2008.

## Loading the Cranfield collection

First, we will install [our library](https://gitlab.fi.muni.cz/xstefan3/pv211-utils) and load the Cranfield collection.

In [1]:
%%capture
! pip install git+https://github.com/MIR-MU/pv211-utils.git

### Loading the documents

Next, we will define a class named `Document` that will represent a preprocessed document from the Cranfield collection. Tokenization and preprocessing of the `title` and `body` attributes of the individual documents as well as the creative use of the `authors`, `bibliography`, and `title` attributes is left to your imagination and craftsmanship.

In [2]:
from pv211_utils.cranfield.entities import CranfieldDocumentBase

class Document(CranfieldDocumentBase):
    """
    A preprocessed Cranfield collection document.

    Parameters
    ----------
    document_id : str
        A unique identifier of the document.
    authors : list of str
        A unique identifiers of the authors of the document.
    bibliography : str
        The bibliographical entry for the document.
    title : str
        The title of the document.
    body : str
        The abstract of the document.

    """
    def __init__(self, document_id: str, authors: str, bibliography: str, title: str, body: str):
        super().__init__(document_id, authors, bibliography, title, body)
    
    def __str__(self):
        return f"{self.title} {self.body}"

We will load documents into the `documents` [ordered dictionary](https://docs.python.org/3.8/library/collections.html#collections.OrderedDict). Each document is an instance of the `Document` class that we have just defined.

In [3]:
from pv211_utils.datasets import CranfieldDataset

cranfield = CranfieldDataset()

documents = cranfield.load_documents(Document)

/home/jovyan/.local/lib/python3.8/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
print('\n'.join(repr(document) for document in list(documents.values())[:3]))
print('...')
print('\n'.join(repr(document) for document in list(documents.values())[-3:]))

<Document 1 “experimental investigation of the aerodynamics of  ...”>
<Document 2 “simple shear flow past a flat plate in an incompre ...”>
<Document 3 “the boundary layer in simple shear flow past a fla ...”>
...
<Document 1398 “stability of rectangular plates under shear and be ...”>
<Document 1399 “buckling of transverse stiffened plates under shea ...”>
<Document 1400 “the buckling shear stress of simply-supported infi ...”>


In [5]:
document = documents['14']
document

<Document 14 “piston theory - a new aerodynamic tool for the aer ...”>

In [6]:
print(document.authors)

ashley,h. and zartarian,g.


In [7]:
print(document.bibliography)

j. ae. scs. 23, 1956, 1109.


In [8]:
print(document.title)

piston theory - a new aerodynamic tool for the aeroelastician .


In [9]:
print(document.body)

piston theory - a new aerodynamic tool for the aeroelastician .   representative applications are described which illustrate the extent to which simplifications in the solutions of high-speed unsteady aeroelastic problems can be achieved through the use of certain aerodynamic techniques known collectively as /piston theory ./  based on a physical model originally proposed by hayes and lighthill, piston theory for airfoils and finite wings has been systematically developed by landahl, utilizing expansions in powers of the thickness ratio and the inverse of the flight mach number m .  when contributions of orders and are negligible, the theory predicts a point-function relationship between the local pressure on the surface of a wing and the normal component of fluid velocity produced by the wing's motion .  the computation of generalized forces in aeroelastic equations, such as the flutter determinant, is then always reduced to elementary integrations of the assumed modes of motion .   e

### Loading the queries
Next, we will define a class named `Query` that will represent a preprocessed query from the Cranfield collection. Tokenization and preprocessing of the `body` attribute of the individual queries is left to your craftsmanship.

In [10]:
from pv211_utils.cranfield.entities import CranfieldQueryBase

class Query(CranfieldQueryBase):
    """
    A preprocessed Cranfield collection query.

    Parameters
    ----------
    query_id : int
        A unique identifier of the query.
    body : str
        The text of the query.

    """
    def __init__(self, query_id: int, body: str):
        super().__init__(query_id, body)
    
    def __str__(self):
        return self.body

We will load queries into the `queries` [ordered dictionary](https://docs.python.org/3.8/library/collections.html#collections.OrderedDict). Each query is an instance of the `Query` class that we have just defined.

In [11]:
queries = cranfield.load_test_queries(Query)

In [12]:
print('\n'.join(repr(query) for query in list(queries.values())[:3]))
print('...')
print('\n'.join(repr(query) for query in list(queries.values())[-3:]))

<Query 12 “how can the aerodynamic performance of channel flo ...”>
<Query 31 “what size of end plate can be safely used to simul ...”>
<Query 84 “references on the methods available for accurately ...”>
...
<Query 16 “can the transverse potential flow about a body of  ...”>
<Query 78 “has anyone explained the kink in the surge line of ...”>
<Query 146 “does a membrane theory exist by which the behaviou ...”>


In [13]:
query = queries[14]
query

<Query 14 “papers on shock-sound wave interaction .”>

In [14]:
print(query.body)

papers on shock-sound wave interaction .


# IR-System

You can try preprocessing and systems that are already in `pv211_utils`, but feel free to implement your own.


In [15]:
from pv211_utils.systems import TfidfSystem
from pv211_utils.preprocessing.preprocessing import SimpleDocPreprocessing

system = TfidfSystem(documents, SimpleDocPreprocessing())

Building the TF-IDF index: 100%|██████████| 1400/1400 [00:00<00:00, 2542.12it/s]


## Evaluation
Finally, we will evaluate your information retrieval system using [the Mean Average Precision](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision) (MAP) evaluation measure.

In [16]:
from pv211_utils.cranfield.loader import load_judgements
from pv211_utils.cranfield.leaderboard import CranfieldLeaderboard
from pv211_utils.cranfield.eval import CranfieldEvaluation

submit_result = False
author_name = 'Surname, Name'

evaluation = CranfieldEvaluation(system, load_judgements(queries, documents),
                                 leaderboard=CranfieldLeaderboard(), author_name=author_name)
evaluation.evaluate(queries, submit_result)


Your system achieved **38.46% MAP score**.

Congratulations, you passed the **22%** minimum! 🥳

Set `submit_result = True` and write your name to the `author_name` variable to submit your result to [the leaderboard](https://docs.google.com/spreadsheets/d/e/2PACX-1vSXuOTclZfHWYxh2rf7hfMeLvcCuE5UsJu7BzteyunhPw3z4YNZjCovjmMB6SnDdgjGyenOgdochaEq/pubhtml). 🏆

The best submissions on the leaderboard will receive *small awards during the semester*, and some *__seriously big__ awards* after the personal check at the end of the competition (2023-03-13). Please be polite, do not spoil the game for the others, and **have fun!** 😉